In [1]:
import copy

import numpy as np
import pandas as pd
from collections import Counter, OrderedDict, defaultdict
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from math import floor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import ipdb

import tqdm

# for Kseniia's model
from cotrain import SSLTrainSet
from cotrain import BinaryCoTrainer
from dataloaders.dataloader import *

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

#увеличим дефолтный размер графиков
from pylab import rcParams
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

import tqdm

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
import pandas as pd 

df = pd.DataFrame(columns=['Algorithm', 'Dataset', 'L_Size', 'U_Size', 'T_Size', 'Accuracy'])
print(df)

Empty DataFrame
Columns: [Algorithm, Dataset, L_Size, U_Size, T_Size, Accuracy]
Index: []


### TESTING (order one)

In [3]:
all_datasets = [
    "pendigits", "mnist", "mnist_fashion", "20_newsgroups" 
]

splits = dict.fromkeys(all_datasets)

splits["pendigits"] = [[20, 1600 , 200], [30, 1600, 200], [50, 1600, 200], [100, 1600, 200],
                       [30, 500, 200], [30, 900, 200], [30, 1300, 200],[30, 1700, 200]]

splits["mnist"] = [[20, 1500, 200], [30, 1500, 200], [50, 1500, 200], [100, 1500, 200],
                   [25, 500, 200], [25, 900, 200], [25, 1300, 200],[25, 1700, 200]]

splits["mnist_fashion"] = [[20, 2000, 1000], [30, 2000, 1000], [50, 2000, 1000], [100, 2000, 1000],
                           [50, 1000, 1000],[50, 1500, 1000], [50, 2000, 1000], [50, 2500, 1000]]

splits["20_newsgroups"] = [[20, 800, 100], [50, 800, 100], [70, 800, 100], [100, 800, 100],
                           [25, 300, 100],[25, 500, 100], [25, 700, 100], [25, 900, 100]]


bin_dict = dict.fromkeys(all_datasets)
bin_dict["pendigits"] = [[4, 9]]
bin_dict["mnist"] = [[4, 9]]
bin_dict["mnist_fashion"] = [[0, 1]]
bin_dict["20_newsgroups"] = [[17, 18]]


random_states = list(range(0, 101, 10))

rf_params = {'n_estimators': 50}

In [4]:
for dataset in all_datasets:
    for binary in bin_dict.get(dataset):
        for split in splits.get(dataset):
            accuracy_list = np.zeros(len(random_states))
            accuracy_list_cp = np.zeros(len(random_states))
            accuracy_list_random = np.zeros(len(random_states))            
            for idx, random_state in enumerate(random_states):
                print("\nDataset : " + dataset," Binary : " + str(binary), " Split : " +str(split))
                X1, Y1, X1u, X2, Y2, X2u, Xtest,Ytest  = get_samples(dataset=dataset,
                                                                     split=split,
                                                                     random_state=random_state,
                                                                     binary=binary,
                                                                     model='co-train')
                
                rf_params['random_state'] = random_state

                co_train = BinaryCoTrainer([RandomForestClassifier(**rf_params) for _ in range(2)])

                train_sets = [SSLTrainSet(X1, Y1, X1u), SSLTrainSet(X2, Y2, X2u)]
                co_train.fit(train_sets)

                co_scores = []
                for pred in co_train.predict(Xtest):
                    co_scores.append(accuracy_score(pred, Ytest))
                
                accuracy_list[idx] = np.mean(co_scores)

                #baseline
                model_bl1 = RandomForestClassifier(**rf_params)
                model_bl2 = RandomForestClassifier(**rf_params)        

                model_bl1.fit(X1, Y1)
                model_bl2.fit(X2, Y2)

                model_bl1_accuracy = accuracy_score(model_bl1.predict(Xtest), Ytest)
                model_bl2_accuracy = accuracy_score(model_bl2.predict(Xtest), Ytest)
                
                accuracy_list_cp[idx] = np.mean((model_bl1_accuracy,model_bl2_accuracy))
            
            df = df.append({'Algorithm':'co-train', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list)}, ignore_index=True)
            df = df.append({'Algorithm':'co-train baseline', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list_cp)}, ignore_index=True)  


Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of labeled data (10, 16)
Size of unlabeled data (800, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of labeled data (10, 16)
Size of unlabeled data (800, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of labeled data (10, 16)
Size of unlabeled data (800, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of labeled data (10, 16)
Size of unlabeled data (800, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of labeled data (10, 16)
Size of unlabeled data (800, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of labeled data (10, 16)
Size of unlabeled data (800, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [20, 1600, 200]
Size of label


Dataset : pendigits  Binary : [4, 9]  Split : [30, 500, 200]
Size of labeled data (15, 16)
Size of unlabeled data (250, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 900, 200]
Size of labeled data (15, 16)
Size of unlabeled data (450, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 900, 200]
Size of labeled data (15, 16)
Size of unlabeled data (450, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 900, 200]
Size of labeled data (15, 16)
Size of unlabeled data (450, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 900, 200]
Size of labeled data (15, 16)
Size of unlabeled data (450, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 900, 200]
Size of labeled data (15, 16)
Size of unlabeled data (450, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 900, 200]
Size of labeled data


Dataset : mnist  Binary : [4, 9]  Split : [30, 1500, 200]
Size of labeled data (15, 784)
Size of unlabeled data (750, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [30, 1500, 200]
Size of labeled data (15, 784)
Size of unlabeled data (750, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [50, 1500, 200]
Size of labeled data (25, 784)
Size of unlabeled data (750, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [50, 1500, 200]
Size of labeled data (25, 784)
Size of unlabeled data (750, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [50, 1500, 200]
Size of labeled data (25, 784)
Size of unlabeled data (750, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [50, 1500, 200]
Size of labeled data (25, 784)
Size of unlabeled data (750, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [50, 1500, 200]
Size of labeled data (2


Dataset : mnist  Binary : [4, 9]  Split : [25, 1300, 200]
Size of labeled data (12, 784)
Size of unlabeled data (650, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [25, 1300, 200]
Size of labeled data (12, 784)
Size of unlabeled data (650, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [25, 1300, 200]
Size of labeled data (12, 784)
Size of unlabeled data (650, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [25, 1700, 200]
Size of labeled data (12, 784)
Size of unlabeled data (850, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [25, 1700, 200]
Size of labeled data (12, 784)
Size of unlabeled data (850, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [25, 1700, 200]
Size of labeled data (12, 784)
Size of unlabeled data (850, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [25, 1700, 200]
Size of labeled data (1


Dataset : mnist_fashion  Binary : [0, 1]  Split : [100, 2000, 1000]
Size of labeled data (50, 784)
Size of unlabeled data (1000, 784)
Size of tets data (1000, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [100, 2000, 1000]
Size of labeled data (50, 784)
Size of unlabeled data (1000, 784)
Size of tets data (1000, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [100, 2000, 1000]
Size of labeled data (50, 784)
Size of unlabeled data (1000, 784)
Size of tets data (1000, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [100, 2000, 1000]
Size of labeled data (50, 784)
Size of unlabeled data (1000, 784)
Size of tets data (1000, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [100, 2000, 1000]
Size of labeled data (50, 784)
Size of unlabeled data (1000, 784)
Size of tets data (1000, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [100, 2000, 1000]
Size of labeled data (50, 784)
Size of unlabeled data (1000, 784)
Size of tets data (1000, 784)

Dataset :


Dataset : 20_newsgroups  Binary : [17, 18]  Split : [20, 800, 100]
Size of labeled data (10, 1921)
Size of unlabeled data (400, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [20, 800, 100]
Size of labeled data (10, 1921)
Size of unlabeled data (400, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [20, 800, 100]
Size of labeled data (10, 1921)
Size of unlabeled data (400, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [20, 800, 100]
Size of labeled data (10, 1921)
Size of unlabeled data (400, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [20, 800, 100]
Size of labeled data (10, 1921)
Size of unlabeled data (400, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [20, 800, 100]
Size of labeled data (10, 1921)
Size of unlabeled data (400, 1921)
Size of tets data (100, 1921)

Dataset :


Dataset : 20_newsgroups  Binary : [17, 18]  Split : [25, 300, 100]
Size of labeled data (12, 1921)
Size of unlabeled data (150, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [25, 300, 100]
Size of labeled data (12, 1921)
Size of unlabeled data (150, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [25, 300, 100]
Size of labeled data (12, 1921)
Size of unlabeled data (150, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [25, 300, 100]
Size of labeled data (12, 1921)
Size of unlabeled data (150, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [25, 300, 100]
Size of labeled data (12, 1921)
Size of unlabeled data (150, 1921)
Size of tets data (100, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [25, 500, 100]
Size of labeled data (12, 1921)
Size of unlabeled data (250, 1921)
Size of tets data (100, 1921)

Dataset :

In [5]:
df.to_csv('results/MYALGO_TEST1.csv')

## VERY FEW LABELED DATA

In [3]:
all_datasets = ["pendigits", "mnist", "mnist_fashion", "20_newsgroups", "STL10", "cifar10"]

splits = dict.fromkeys(all_datasets)

splits["pendigits"] = [[4, 600, 200], [8, 600, 200], [12, 600, 200], [16, 600, 200]]
splits["mnist"] = [[4, 600, 200], [8, 600, 200], [12, 600, 200], [16, 600, 200]]
splits["mnist_fashion"] = [[4, 600, 200], [8, 600, 200], [12, 600, 200], [16, 600, 200]]
splits["20_newsgroups"] = [[4, 600, 200], [8, 600, 200], [12, 600, 200], [16, 600, 200]]
splits["STL10"] = [[4, 600, 200], [8, 600, 200], [12, 600, 200], [16, 600, 200]]
splits["cifar10"] = [[4, 600, 200], [8, 600, 200], [12, 600, 200], [16, 600, 200]]

bin_dict = dict.fromkeys(all_datasets)
bin_dict["pendigits"] = [[4, 9]]
bin_dict["mnist"] = [[4, 9]]
bin_dict["mnist_fashion"] = [[0, 1]]
bin_dict["20_newsgroups"] = [[17, 18]]
bin_dict["STL10"] = [[1,2], [1,9]]
bin_dict["cifar10"] = [[1,2], [1,9]]


random_states = list(range(0, 101, 10))

rf_params = {'n_estimators': 50}

df_s = pd.DataFrame(columns=['Algorithm', 'Dataset', 'L_Size', 'U_Size', 'T_Size', 'Accuracy'])

In [4]:
for dataset in all_datasets:
    for binary in bin_dict.get(dataset):
        for split in splits.get(dataset):
            accuracy_list = np.zeros(len(random_states))
            accuracy_list_cp = np.zeros(len(random_states))
            accuracy_list_random = np.zeros(len(random_states))            
            for idx, random_state in enumerate(random_states):
                print("\nDataset : " + dataset," Binary : " + str(binary), " Split : " +str(split))
                X1, Y1, X1u, X2, Y2, X2u, Xtest,Ytest  = get_samples(dataset=dataset,
                                                                     split=split,
                                                                     random_state=random_state,
                                                                     binary=binary,
                                                                     model='co-train')
                
                rf_params['random_state'] = random_state

                co_train = BinaryCoTrainer([RandomForestClassifier(**rf_params) for _ in range(2)])

                train_sets = [SSLTrainSet(X1, Y1, X1u), SSLTrainSet(X2, Y2, X2u)]
                co_train.fit(train_sets)

                co_scores = []
                for pred in co_train.predict(Xtest):
                    co_scores.append(accuracy_score(pred, Ytest))
                
                accuracy_list[idx] = np.mean(co_scores)

                #baseline
                model_bl1 = RandomForestClassifier(**rf_params)
                model_bl2 = RandomForestClassifier(**rf_params)        

                model_bl1.fit(X1, Y1)
                model_bl2.fit(X2, Y2)

                model_bl1_accuracy = accuracy_score(model_bl1.predict(Xtest), Ytest)
                model_bl2_accuracy = accuracy_score(model_bl2.predict(Xtest), Ytest)
                
                accuracy_list_cp[idx] = np.mean((model_bl1_accuracy,model_bl2_accuracy))
            
            df_s = df_s.append({'Algorithm':'co-train', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list)}, ignore_index=True)
            df_s = df_s.append({'Algorithm':'co-train baseline', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list_cp)}, ignore_index=True)  


Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size of unlabeled data (300, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size of unlabeled data (300, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size of unlabeled data (300, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size of unlabeled data (300, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size of unlabeled data (300, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size of unlabeled data (300, 16)
Size of tets data (200, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [4, 600, 200]
Size of labeled data (2, 16)
Size


Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist  Binary : [4, 9]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlab


Dataset : mnist_fashion  Binary : [0, 1]  Split : [8, 600, 200]
Size of labeled data (4, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [12, 600, 200]
Size of labeled data (6, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [12, 600, 200]
Size of labeled data (6, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [12, 600, 200]
Size of labeled data (6, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [12, 600, 200]
Size of labeled data (6, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [12, 600, 200]
Size of labeled data (6, 784)
Size of unlabeled data (300, 784)
Size of tets data (200, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Spli


Dataset : 20_newsgroups  Binary : [17, 18]  Split : [12, 600, 200]
Size of labeled data (6, 1921)
Size of unlabeled data (300, 1921)
Size of tets data (200, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [12, 600, 200]
Size of labeled data (6, 1921)
Size of unlabeled data (300, 1921)
Size of tets data (200, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [12, 600, 200]
Size of labeled data (6, 1921)
Size of unlabeled data (300, 1921)
Size of tets data (200, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [12, 600, 200]
Size of labeled data (6, 1921)
Size of unlabeled data (300, 1921)
Size of tets data (200, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [12, 600, 200]
Size of labeled data (6, 1921)
Size of unlabeled data (300, 1921)
Size of tets data (200, 1921)

Dataset : 20_newsgroups  Binary : [17, 18]  Split : [16, 600, 200]
Size of labeled data (8, 1921)
Size of unlabeled data (300, 1921)
Size of tets data (200, 1921)

Dataset : 20_ne


Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]
Size of labeled data (8, 27648)
Size of unlabeled data (300, 27648)
Size of tets data (200, 27648)

Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]
Size of labeled data (8, 27648)
Size of unlabeled data (300, 27648)
Size of tets data (200, 27648)

Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]
Size of labeled data (8, 27648)
Size of unlabeled data (300, 27648)
Size of tets data (200, 27648)

Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]
Size of labeled data (8, 27648)
Size of unlabeled data (300, 27648)
Size of tets data (200, 27648)

Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]
Size of labeled data (8, 27648)
Size of unlabeled data (300, 27648)
Size of tets data (200, 27648)

Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]
Size of labeled data (8, 27648)
Size of unlabeled data (300, 27648)
Size of tets data (200, 27648)

Dataset : STL10  Binary : [1, 2]  Split : [16, 600, 200]



Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 2]  Split : [4, 600, 200]
Size of lab


Dataset : cifar10  Binary : [1, 9]  Split : [4, 600, 200]
Size of labeled data (2, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 9]  Split : [8, 600, 200]
Size of labeled data (4, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 9]  Split : [8, 600, 200]
Size of labeled data (4, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 9]  Split : [8, 600, 200]
Size of labeled data (4, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 9]  Split : [8, 600, 200]
Size of labeled data (4, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 9]  Split : [8, 600, 200]
Size of labeled data (4, 3072)
Size of unlabeled data (300, 3072)
Size of tets data (200, 3072)

Dataset : cifar10  Binary : [1, 9]  Split : [8, 600, 200]
Size of lab

In [6]:
df_s.to_csv('results/MYALGO_TEST1_VERYSMALL.csv')